In [1]:
from rdkit import RDLogger
lg = RDLogger.logger()
lg.setLevel(4)

In [2]:
from __future__ import print_function
from rdkit.Chem.Draw import IPythonConsole, ReactionToImage, MolToImage, MolsToGridImage
from IPython.display import SVG, display, clear_output
import rdkit.Chem as Chem
import rdkit.Chem.AllChem as AllChem
from rdkit import DataStructs
import pandas as pd
import numpy as np
from tqdm import tqdm
import json
import sys
sys.path.append('../../')
from retrosim.utils.draw import ReactionStringToImage, TransformStringToImage
from retrosim.utils.generate_retro_templates import process_an_example
from retrosim.data.get_data import get_data_df, split_data_df
from rdchiral.main import rdchiralRun, rdchiralReaction, rdchiralReactants

from joblib import Parallel, delayed
import multiprocessing
num_cores = multiprocessing.cpu_count()

In [3]:
data = get_data_df('../data/data_processed.csv')
split_data_df(data) # 80/10/10 within each class

15150 rows with class value 1
11893 rows with class value 2
5661 rows with class value 3
909 rows with class value 4
672 rows with class value 5
8237 rows with class value 6
4613 rows with class value 7
811 rows with class value 8
1834 rows with class value 9
230 rows with class value 10
train    40004
test      5006
val       5000
Name: dataset, dtype: int64


## Select one set of settings to test
(use the test_similiarity.py script to do the full set of experiments)

In [4]:
class_ = 1
similarity_metric = DataStructs.BulkTanimotoSimilarity # BulkDiceSimilarity or BulkTanimotoSimilarity
similarity_label = 'Tanimoto'
getfp = lambda smi: AllChem.GetMorganFingerprint(Chem.MolFromSmiles(smi), 2, useFeatures=False)
getfp_label = 'Morgan2noFeat'
dataset = 'val'
label = '{}_class{}_fp{}_sim{}'.format(
    dataset,
    class_,
    getfp_label,
    similarity_label,
)

### Only get new FPs if necessary - is a little slow

In [5]:
try:
    if prev_FP != getfp:
        raise NameError
except NameError:
    all_fps = []
    for smi in tqdm(data['prod_smiles']):
        all_fps.append(getfp(smi))
    data['prod_fp'] = all_fps
    prev_FP = getfp

100%|██████████████████████████████████████████████████████████████████████████| 50010/50010 [00:23<00:00, 2110.79it/s]


### Get the training data subset of the full data

In [6]:
if class_ != 'all':
    datasub = data.loc[data['class'] == class_].loc[data['dataset'] == 'train']
  
    datasub_val = data.loc[data['class'] == class_].loc[data['dataset'] == dataset]
else:
    datasub = data.loc[data['dataset'] == 'train']
    datasub_val = data.loc[data['dataset'] == dataset]
fps = list(datasub['prod_fp'])
print('Size of knowledge base: {}'.format(len(fps)))

Size of knowledge base: 12120


## Go through full validation/test data, define the function for processing

In [7]:
jx_cache = {}
draw = False
debug = False

def do_one(ix, draw=draw, debug=debug, max_prec=100):
    
    rec_for_printing = ''
    
    ex = Chem.MolFromSmiles(datasub_val['prod_smiles'][ix])
    rct = rdchiralReactants(datasub_val['prod_smiles'][ix])
    if draw: 
        print('Mol {}'.format(ix))
    if debug:
        rec_for_printing += datasub_val['prod_smiles'][ix] + '\n'
        rec_for_printing += 'True reaction:'
        rec_for_printing += datasub_val['rxn_smiles'][ix] + '\n'
    fp = datasub_val['prod_fp'][ix]
    
    sims = similarity_metric(fp, [fp_ for fp_ in datasub['prod_fp']])
    js = np.argsort(sims)[::-1]

    if draw: display(ReactionStringToImage(datasub_val['rxn_smiles'][ix]))
    
    prec_goal = Chem.MolFromSmiles(datasub_val['rxn_smiles'][ix].split('>')[0])
    [a.ClearProp('molAtomMapNumber') for a in prec_goal.GetAtoms()]
    prec_goal = Chem.MolToSmiles(prec_goal, True)
    
    # Sometimes stereochem takes another canonicalization...
    prec_goal = Chem.MolToSmiles(Chem.MolFromSmiles(prec_goal), True)
    if debug:
        rec_for_printing += 'prec_goal: {}\n'.format(prec_goal)
    # Get probability of precursors
    probs = {}
    
    for ji, j in enumerate(js[:max_prec]):
        jx = datasub.index[j]
        
        if draw:
            print('\n\n' + '-'*50 + '\n')
            print('RANK {} PRECEDENT'.format(ji+1))
            print('PRODUCT MATCH SCORE: {}'.format(sims[j]))
            display(ReactionStringToImage(datasub['rxn_smiles'][jx]))
        if debug:
            rec_for_printing += '\nReaction precedent {}, prod similarity {}\n'.format(
                ji+1, sims[j])
            rec_for_printing += '-> rxn_smiles {}\n'.format(datasub['rxn_smiles'][jx])
        if jx in jx_cache:
            (rxn, template, rcts_ref_fp) = jx_cache[jx]
        else:
            retro_canonical = process_an_example(datasub['rxn_smiles'][jx], super_general=True)
            if retro_canonical is None:
                continue
            template = '(' + retro_canonical.replace('>>', ')>>')
            rcts_ref_fp = getfp(datasub['rxn_smiles'][jx].split('>')[0])
            rxn = rdchiralReaction(template)
            jx_cache[jx] = (rxn, template, rcts_ref_fp)
            
        if debug:
            rec_for_printing += '-> template: {}\n'.format(template)
        if draw:
            print('-> template: {}'.format(template))
            
        try:
            outcomes = rdchiralRun(rxn, rct, combine_enantiomers=False)
        except Exception as e:
            print(e)
            outcomes = []

        if not outcomes and draw:
            print('No precursors could be generated!')
            
        for precursors in outcomes:
            precursors_fp = getfp(precursors)
            precursors_sim = similarity_metric(precursors_fp, [rcts_ref_fp])[0]
            if debug:
                rec_for_printing += 'prec sim {} smiles {}\n'.format(precursors_sim, precursors)
            if draw:
                print('Precursor similarity {}, overall {}, smiles {}'.format(
                    precursors_sim, precursors_sim*sims[j], precursors))
                display(MolToImage(Chem.MolFromSmiles(precursors)))
            if precursors in probs:
                probs[precursors] = max(probs[precursors], precursors_sim * sims[j])
            else:
                probs[precursors] = precursors_sim * sims[j]
    
    testlimit = 50
    mols = []
    legends = []

    found_rank = 9999
    for r, (prec, prob) in enumerate(sorted(probs.items(), key=lambda x:x[1], reverse=True)[:testlimit]):
        mols.append(Chem.MolFromSmiles(prec))
        if prec == prec_goal:
            found_rank = r + 1
            legends.append('[TRUE] {}'.format(prob))
        else:
            legends.append('{}'.format(prob))
            pass
    if found_rank == 9999 and debug:
        print(datasub_val['rxn_smiles'][ix])
        display(ReactionStringToImage(datasub_val['rxn_smiles'][ix]))
        print(prec_goal)
        draw = True
    
    if draw: 
        img=MolsToGridImage(mols[:9],molsPerRow=3,subImgSize=(300,300),legends=legends[:9])
        display(img)
        for mol in mols[:9]:
            print(Chem.MolToSmiles(mol, True))
    if debug:
        print(rec_for_printing)
    if draw or debug:
        raw_input('pause')
        clear_output()
        
    return found_rank

### Code for running one-at-a-time on a single core

In [8]:
found_at_rank = []
from collections import defaultdict
found_at_rank_counter = defaultdict(int)

for ii, ix in enumerate(datasub_val.index):
        
    if ii % 1 == 0 and ii > 0:
        print('{} done'.format(ii))
        print('Current found_at_rank: {}'.format(found_at_rank_counter))
        
    found_rank = do_one(ix)
    found_at_rank.append(found_rank)
    found_at_rank_counter[found_rank] += 1

1 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 1})
2 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 2})
3 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 2, 1: 1})
4 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 2, 1: 2})
5 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 2, 1: 2, 6: 1})
6 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 2, 1: 3, 6: 1})
7 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 3, 1: 3, 6: 1})
8 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 3, 1: 4, 6: 1})
9 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 3, 1: 5, 6: 1})
10 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 3, 1: 5, 6: 1, 9999: 1})
11 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 3, 1: 6, 6: 1, 9999: 1})
12 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 3, 1: 6, 6: 1, 9999: 2})
13 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 3, 1: 7, 6: 1, 9999:

70 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 5, 1: 26, 6: 3, 9999: 7, 20: 1, 4: 6, 5: 3, 3: 8, 33: 1, 7: 6, 9: 2, 12: 1, 21: 1})
71 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 5, 1: 27, 6: 3, 9999: 7, 20: 1, 4: 6, 5: 3, 3: 8, 33: 1, 7: 6, 9: 2, 12: 1, 21: 1})
72 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 5, 1: 28, 6: 3, 9999: 7, 20: 1, 4: 6, 5: 3, 3: 8, 33: 1, 7: 6, 9: 2, 12: 1, 21: 1})
73 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 5, 1: 29, 6: 3, 9999: 7, 20: 1, 4: 6, 5: 3, 3: 8, 33: 1, 7: 6, 9: 2, 12: 1, 21: 1})
74 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 5, 1: 30, 6: 3, 9999: 7, 20: 1, 4: 6, 5: 3, 3: 8, 33: 1, 7: 6, 9: 2, 12: 1, 21: 1})
75 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 5, 1: 31, 6: 3, 9999: 7, 20: 1, 4: 6, 5: 3, 3: 8, 33: 1, 7: 6, 9: 2, 12: 1, 21: 1})
76 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 5, 1: 32, 6: 3, 9999: 7, 20: 1, 4: 6, 5: 3, 3: 8, 33: 1, 7: 6,

123 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 9, 1: 52, 6: 3, 9999: 11, 20: 1, 4: 10, 5: 7, 3: 14, 33: 1, 7: 6, 9: 2, 12: 2, 21: 1, 10: 2, 35: 1, 16: 1})
124 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 9, 1: 52, 6: 3, 9999: 11, 20: 1, 4: 10, 5: 8, 3: 14, 33: 1, 7: 6, 9: 2, 12: 2, 21: 1, 10: 2, 35: 1, 16: 1})
125 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 9, 1: 52, 6: 3, 9999: 11, 20: 1, 4: 10, 5: 8, 3: 14, 33: 1, 7: 6, 9: 2, 12: 2, 21: 1, 10: 2, 35: 1, 16: 1, 15: 1})
126 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 10, 1: 52, 6: 3, 9999: 11, 20: 1, 4: 10, 5: 8, 3: 14, 33: 1, 7: 6, 9: 2, 12: 2, 21: 1, 10: 2, 35: 1, 16: 1, 15: 1})
127 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 10, 1: 52, 6: 4, 9999: 11, 20: 1, 4: 10, 5: 8, 3: 14, 33: 1, 7: 6, 9: 2, 12: 2, 21: 1, 10: 2, 35: 1, 16: 1, 15: 1})
128 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 10, 1: 53, 6: 4, 9999: 11, 20: 1, 4: 10, 5: 8, 3: 14, 33: 1

169 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 17, 1: 75, 6: 4, 9999: 13, 20: 1, 4: 12, 5: 9, 3: 16, 33: 1, 7: 8, 9: 2, 12: 2, 21: 1, 10: 4, 35: 1, 16: 1, 15: 1, 8: 1})
170 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 17, 1: 76, 6: 4, 9999: 13, 20: 1, 4: 12, 5: 9, 3: 16, 33: 1, 7: 8, 9: 2, 12: 2, 21: 1, 10: 4, 35: 1, 16: 1, 15: 1, 8: 1})
171 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 17, 1: 76, 6: 4, 9999: 13, 20: 1, 4: 12, 5: 10, 3: 16, 33: 1, 7: 8, 9: 2, 12: 2, 21: 1, 10: 4, 35: 1, 16: 1, 15: 1, 8: 1})
172 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 17, 1: 76, 6: 4, 9999: 13, 20: 1, 4: 13, 5: 10, 3: 16, 33: 1, 7: 8, 9: 2, 12: 2, 21: 1, 10: 4, 35: 1, 16: 1, 15: 1, 8: 1})
173 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 18, 1: 76, 6: 4, 9999: 13, 20: 1, 4: 13, 5: 10, 3: 16, 33: 1, 7: 8, 9: 2, 12: 2, 21: 1, 10: 4, 35: 1, 16: 1, 15: 1, 8: 1})
174 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 18, 1: 76

214 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 24, 1: 95, 6: 4, 9999: 18, 20: 1, 4: 16, 5: 10, 3: 19, 33: 1, 7: 9, 9: 4, 12: 3, 21: 1, 10: 4, 35: 1, 16: 2, 15: 1, 8: 1})
215 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 24, 1: 96, 6: 4, 9999: 18, 20: 1, 4: 16, 5: 10, 3: 19, 33: 1, 7: 9, 9: 4, 12: 3, 21: 1, 10: 4, 35: 1, 16: 2, 15: 1, 8: 1})
216 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 24, 1: 97, 6: 4, 9999: 18, 20: 1, 4: 16, 5: 10, 3: 19, 33: 1, 7: 9, 9: 4, 12: 3, 21: 1, 10: 4, 35: 1, 16: 2, 15: 1, 8: 1})
217 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 24, 1: 97, 6: 4, 9999: 18, 20: 1, 4: 16, 5: 11, 3: 19, 33: 1, 7: 9, 9: 4, 12: 3, 21: 1, 10: 4, 35: 1, 16: 2, 15: 1, 8: 1})
218 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 24, 1: 97, 6: 4, 9999: 18, 20: 1, 4: 16, 5: 11, 3: 19, 33: 1, 7: 10, 9: 4, 12: 3, 21: 1, 10: 4, 35: 1, 16: 2, 15: 1, 8: 1})
219 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 24, 1:

...

1380 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 181, 1: 655, 6: 34, 9999: 108, 20: 4, 4: 73, 5: 49, 3: 92, 33: 1, 7: 37, 9: 26, 12: 17, 21: 6, 10: 11, 35: 1, 16: 6, 15: 6, 8: 24, 22: 1, 14: 8, 11: 15, 28: 2, 24: 1, 13: 11, 30: 1, 17: 4, 18: 3, 29: 2, 19: 1})
1381 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 181, 1: 656, 6: 34, 9999: 108, 20: 4, 4: 73, 5: 49, 3: 92, 33: 1, 7: 37, 9: 26, 12: 17, 21: 6, 10: 11, 35: 1, 16: 6, 15: 6, 8: 24, 22: 1, 14: 8, 11: 15, 28: 2, 24: 1, 13: 11, 30: 1, 17: 4, 18: 3, 29: 2, 19: 1})
1382 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 182, 1: 656, 6: 34, 9999: 108, 20: 4, 4: 73, 5: 49, 3: 92, 33: 1, 7: 37, 9: 26, 12: 17, 21: 6, 10: 11, 35: 1, 16: 6, 15: 6, 8: 24, 22: 1, 14: 8, 11: 15, 28: 2, 24: 1, 13: 11, 30: 1, 17: 4, 18: 3, 29: 2, 19: 1})
1383 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 183, 1: 656, 6: 34, 9999: 108, 20: 4, 4: 73, 5: 49, 3: 92, 33: 1, 7: 37, 9: 26, 12: 17, 21: 6, 10: 11, 35: 1, 1

1410 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 187, 1: 670, 6: 34, 9999: 109, 20: 5, 4: 75, 5: 49, 3: 95, 33: 1, 7: 38, 9: 26, 12: 17, 21: 6, 10: 12, 35: 1, 16: 6, 15: 6, 8: 24, 22: 1, 14: 8, 11: 15, 28: 2, 24: 1, 13: 11, 30: 1, 17: 4, 18: 3, 29: 2, 19: 1})
1411 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 187, 1: 671, 6: 34, 9999: 109, 20: 5, 4: 75, 5: 49, 3: 95, 33: 1, 7: 38, 9: 26, 12: 17, 21: 6, 10: 12, 35: 1, 16: 6, 15: 6, 8: 24, 22: 1, 14: 8, 11: 15, 28: 2, 24: 1, 13: 11, 30: 1, 17: 4, 18: 3, 29: 2, 19: 1})
1412 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 187, 1: 672, 6: 34, 9999: 109, 20: 5, 4: 75, 5: 49, 3: 95, 33: 1, 7: 38, 9: 26, 12: 17, 21: 6, 10: 12, 35: 1, 16: 6, 15: 6, 8: 24, 22: 1, 14: 8, 11: 15, 28: 2, 24: 1, 13: 11, 30: 1, 17: 4, 18: 3, 29: 2, 19: 1})
1413 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 187, 1: 673, 6: 34, 9999: 109, 20: 5, 4: 75, 5: 49, 3: 95, 33: 1, 7: 38, 9: 26, 12: 17, 21: 6, 10: 12, 35: 1, 1

1440 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 192, 1: 683, 6: 35, 9999: 109, 20: 5, 4: 79, 5: 49, 3: 101, 33: 1, 7: 39, 9: 26, 12: 17, 21: 6, 10: 12, 35: 1, 16: 6, 15: 6, 8: 24, 22: 1, 14: 8, 11: 15, 28: 2, 24: 1, 13: 11, 30: 1, 17: 4, 18: 3, 29: 2, 19: 1})
1441 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 192, 1: 683, 6: 35, 9999: 110, 20: 5, 4: 79, 5: 49, 3: 101, 33: 1, 7: 39, 9: 26, 12: 17, 21: 6, 10: 12, 35: 1, 16: 6, 15: 6, 8: 24, 22: 1, 14: 8, 11: 15, 28: 2, 24: 1, 13: 11, 30: 1, 17: 4, 18: 3, 29: 2, 19: 1})
1442 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 193, 1: 683, 6: 35, 9999: 110, 20: 5, 4: 79, 5: 49, 3: 101, 33: 1, 7: 39, 9: 26, 12: 17, 21: 6, 10: 12, 35: 1, 16: 6, 15: 6, 8: 24, 22: 1, 14: 8, 11: 15, 28: 2, 24: 1, 13: 11, 30: 1, 17: 4, 18: 3, 29: 2, 19: 1})
1443 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 193, 1: 683, 6: 35, 9999: 111, 20: 5, 4: 79, 5: 49, 3: 101, 33: 1, 7: 39, 9: 26, 12: 17, 21: 6, 10: 12, 35: 

1470 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 196, 1: 699, 6: 37, 9999: 113, 20: 5, 4: 79, 5: 50, 3: 104, 33: 1, 7: 39, 9: 26, 12: 17, 21: 6, 10: 12, 35: 1, 16: 6, 15: 6, 8: 24, 22: 1, 14: 8, 11: 15, 28: 2, 24: 1, 13: 11, 30: 1, 17: 4, 18: 3, 29: 2, 19: 1})
1471 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 196, 1: 699, 6: 38, 9999: 113, 20: 5, 4: 79, 5: 50, 3: 104, 33: 1, 7: 39, 9: 26, 12: 17, 21: 6, 10: 12, 35: 1, 16: 6, 15: 6, 8: 24, 22: 1, 14: 8, 11: 15, 28: 2, 24: 1, 13: 11, 30: 1, 17: 4, 18: 3, 29: 2, 19: 1})
1472 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 196, 1: 700, 6: 38, 9999: 113, 20: 5, 4: 79, 5: 50, 3: 104, 33: 1, 7: 39, 9: 26, 12: 17, 21: 6, 10: 12, 35: 1, 16: 6, 15: 6, 8: 24, 22: 1, 14: 8, 11: 15, 28: 2, 24: 1, 13: 11, 30: 1, 17: 4, 18: 3, 29: 2, 19: 1})
1473 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 197, 1: 700, 6: 38, 9999: 113, 20: 5, 4: 79, 5: 50, 3: 104, 33: 1, 7: 39, 9: 26, 12: 17, 21: 6, 10: 12, 35: 

1501 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 197, 1: 716, 6: 42, 9999: 115, 20: 5, 4: 80, 5: 50, 3: 108, 33: 1, 7: 39, 9: 26, 12: 17, 21: 6, 10: 12, 35: 1, 16: 6, 15: 6, 8: 24, 22: 1, 14: 8, 11: 16, 28: 2, 24: 1, 13: 11, 30: 1, 17: 4, 18: 3, 29: 2, 19: 1})
1502 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 197, 1: 716, 6: 43, 9999: 115, 20: 5, 4: 80, 5: 50, 3: 108, 33: 1, 7: 39, 9: 26, 12: 17, 21: 6, 10: 12, 35: 1, 16: 6, 15: 6, 8: 24, 22: 1, 14: 8, 11: 16, 28: 2, 24: 1, 13: 11, 30: 1, 17: 4, 18: 3, 29: 2, 19: 1})
1503 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 197, 1: 717, 6: 43, 9999: 115, 20: 5, 4: 80, 5: 50, 3: 108, 33: 1, 7: 39, 9: 26, 12: 17, 21: 6, 10: 12, 35: 1, 16: 6, 15: 6, 8: 24, 22: 1, 14: 8, 11: 16, 28: 2, 24: 1, 13: 11, 30: 1, 17: 4, 18: 3, 29: 2, 19: 1})
1504 done
Current found_at_rank: defaultdict(<class 'int'>, {2: 198, 1: 717, 6: 43, 9999: 115, 20: 5, 4: 80, 5: 50, 3: 108, 33: 1, 7: 39, 9: 26, 12: 17, 21: 6, 10: 12, 35: 

### Code to run in parallel

In [9]:
# Parallel
inputs = list(datasub_val.index)
found_at_rank = Parallel(n_jobs=num_cores, verbose=5)(delayed(do_one)(i) for i in inputs)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done   3 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done   4 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done   5 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done   6 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done   7 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done   8 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done  10 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done  11 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done  12 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done  13 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done  14 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done  15 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.4s
[Parallel(

PicklingError: Could not pickle the task to send it to the workers.

### Evaluate

In [ ]:
# Evaluate
def ranks_to_acc(found_at_rank, fid=None):
    def fprint(txt):
        print(txt)
        if fid is not None:
            fid.write(txt + '\n')
            
    tot = float(len(found_at_rank))
    fprint('{:>8} \t {:>8}'.format('top-n', 'accuracy'))
    accs = []
    for n in [1, 3, 5, 10, 20, 50]:
        accs.append(sum([r <= n for r in found_at_rank]) / tot)
        fprint('{:>8} \t {:>8}'.format(n, accs[-1]))
    return accs

accs = ranks_to_acc(found_at_rank)

## Example

In [ ]:
inputs = list(datasub_val.index)
do_one(inputs[32], debug=True, draw=True, max_prec=20)

In [ ]:
data = get_data_df('../data/data_processed.csv')
split_data_df(data) # 80/10/10 within each class